In [10]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [11]:
def convert_df_to_numpy(df, scaler):
  y = df['label'].to_numpy().astype(np.int32)
  X = scaler.transform(df.drop(columns=['label']).to_numpy())
  return X, y

center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")

center_surround_scaler = StandardScaler()
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())

center_surround_train_data = convert_df_to_numpy(center_surround_train_df, center_surround_scaler)
center_surround_test_data = convert_df_to_numpy(center_surround_test_df, center_surround_scaler)
center_surround_valid_data = convert_df_to_numpy(center_surround_valid_df, center_surround_scaler)

print(center_surround_train_data[0][:5])
print(center_surround_train_data[1][:5])

[[-1.36846388  1.54362905]
 [ 0.13476095 -0.35808876]
 [-1.17397245  2.0546884 ]
 [ 0.12212715 -0.3837286 ]
 [-0.08095132 -1.53810042]]
[1 0 1 0 1]


In [31]:
INPUT_DIM = 2
OUTPUT_DIM = 1
HIDDEN_NODES = 4
LEARNING_RATE = 0.01

In [37]:
class FeedForwardNeuralNetwork:
  def __init__(self, train_data, test_data, valid_data, input_dim, output_dim, hidden_nodes, learning_rate, hidden_activation="sigmoid", output_activation='sigmoid'):
    self.train_data = train_data
    self.test_data = test_data
    self.valid_data = valid_data
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_nodes = hidden_nodes
    self.learning_rate = learning_rate
    self.hidden_activation = self.sigmoid if hidden_activation == "sigmoid" else None 
    self.output_activation = self.sigmoid if output_activation == "sigmoid" else None 

    self.W_input_hidden = None
    self.W_hidden_output = None
    self.B_input_hidden = None
    self.B_hidden_output = None

    self.initialize_weights_and_biases()
    self.forward()
  
  def sigmoid(self, x):
    return 1/(1 + np.exp(-x))
  
  def softmax(self, x):
    return np.exp(x)/sum(np.exp(x))

  def initialize_weights_and_biases(self):
    """
    Initialize weights and biases for input-hidden and hidden-output layers.

    Shapes:
      Input -> Hidden:  (2, 4) weights, (4,) biases
      Hidden -> Output: (4, 2) weights, (2,) biases
    """
    print("**Entered the weight and biases initialization**")
    self.W_input_hidden = np.random.random((self.input_dim, self.hidden_nodes))
    self.B_input_hidden = np.zeros((self.hidden_nodes,))

    self.W_hidden_output = np.random.random((self.hidden_nodes, self.output_dim))
    self.B_hidden_output = np.zeros((self.output_dim))

    return

  def forward(self):
    """
    Perform forward pass through hidden and output layers with activations.

    Output (first 3 points):
      [[0.69484065]
      [0.74270986]
      [0.72185353]]
    """
    
    print("\n**Entered the forward pass**")

    # Input to hidden layer with activation 
    train_features = self.train_data[0]
    hidden_logits = np.dot(train_features, self.W_input_hidden) + self.B_input_hidden
    hidden_output = self.hidden_activation(hidden_logits)
    
    # The output from the first layer (hidden_output) is now the input into the next layer
    output_logits = np.dot(hidden_output, self.W_hidden_output) + self.B_hidden_output
    output = self.output_activation(output_logits)
    return output
  
  def binary_cross_entropy_loss(self, predictions, actuals):
    """
    UPDATE ME
    """
    loss = 0.0
    for i in range(predictions.shape[0]):
        actual = actuals[i]
        pred = predictions[i]
        internal = actual * np.log(pred) + (1 - actual) * np.log(1 - pred)
        loss += internal  
    return -loss / predictions.shape[0]


FeedForwardNeuralNetwork(
  train_data=center_surround_train_data,
  test_data=center_surround_test_data,
  valid_data=center_surround_valid_data,
  input_dim=INPUT_DIM,
  output_dim=OUTPUT_DIM,
  hidden_nodes=HIDDEN_NODES,
  learning_rate=LEARNING_RATE
)

**Entered the weight and biases initialization**

**Entered the forward pass**
[[0.69484065]
 [0.74270986]
 [0.72185353]
 [0.74122793]
 [0.69835022]
 [0.72956145]
 [0.76160363]
 [0.81369625]
 [0.77199206]
 [0.68348804]
 [0.78074368]
 [0.68143562]
 [0.8237822 ]
 [0.73187005]
 [0.70815021]
 [0.75842824]
 [0.83546116]
 [0.7587434 ]
 [0.72663823]
 [0.7316732 ]
 [0.78422306]
 [0.75746248]
 [0.7902659 ]
 [0.81686218]
 [0.65023317]
 [0.68252027]
 [0.84009811]
 [0.77957029]
 [0.7037367 ]
 [0.77027083]
 [0.75728228]
 [0.80536863]
 [0.60641199]
 [0.69382853]
 [0.75476992]
 [0.70815286]
 [0.62829784]
 [0.62583743]
 [0.74872029]
 [0.61798252]
 [0.8033643 ]
 [0.7212943 ]
 [0.81556583]
 [0.75824362]
 [0.85173041]
 [0.64916072]
 [0.83071876]
 [0.79168904]
 [0.83654859]
 [0.83955405]
 [0.74125749]
 [0.76216256]
 [0.70740951]
 [0.72365679]
 [0.63871447]
 [0.67399582]
 [0.80056372]
 [0.60716506]
 [0.76451817]
 [0.779166  ]
 [0.75662317]
 [0.72956457]
 [0.68433962]
 [0.75760761]
 [0.75315526]
 [0.7358506